In [1]:
#load in functions required and dependencies to use them
%run all_functions.ipynb

In [2]:
#get flow + met data
flowDict = get_ProvinceDailyStreamflow()
precDict = get_ProvinceDailyPrec('AB')
tempDict = get_ProvinceDailyTemp('AB')

In [3]:
#unpack data

stationLat = flowDict['stationLat']
stationLon = flowDict['stationLon']
eraLat = tempDict['latERA']
eraLon = tempDict['lonERA']

flowDays = flowDict['windowDays']
flowMonths = flowDict['windowMonths']
flowYears = flowDict['windowYears']
eraDays = tempDict['daysERA']
eraMonths = tempDict['monthsERA']
eraYears = tempDict['yearsERA']

F = flowDict['all_flowwindow_norm_NF'] #normalized discharge with nans filled (NF)
T = tempDict['T']
P = precDict['P']

In [4]:
#clip to make data have same time range
startYear = max(int(np.min(eraYears)),int(np.min(flowYears)))

indStartERA = min(np.argwhere(eraYears==startYear))[0]
indStartFlow = min(np.argwhere(flowYears==startYear))[0]

F = F[indStartFlow:]
T = T[indStartERA:]
P = P[indStartERA:]

flowDays = flowDays[indStartFlow:]
flowMonths = flowMonths[indStartFlow:]
flowYears = flowYears[indStartFlow:]

eraDays = eraDays[indStartERA:]
eraMonths = eraMonths[indStartERA:]
eraYears = eraYears[indStartERA:]

In [5]:
#prep data: make training and testing sets

#indices of testing/training
trainStartYear = 1987
trainFinYear = 2000
testStartYear = 2001
testFinYear = 2010

trainInds = np.squeeze(np.argwhere((flowYears>=trainStartYear) & (flowYears<=trainFinYear)))
testInds = np.squeeze(np.argwhere((flowYears>testStartYear) & (flowYears<=testFinYear)))

#input

Tmean_train = np.mean([T[trainInds[ii]] for ii in range(len(trainInds))])
Tstd_train = np.std([T[trainInds[ii]] for ii in range(len(trainInds))])
Tnorm = (T - Tmean_train)/Tstd_train

Pmean_train = np.mean([P[trainInds[ii]] for ii in range(len(trainInds))])
Pstd_train = np.std([P[trainInds[ii]] for ii in range(len(trainInds))])
Pnorm = (P - Pmean_train)/Pstd_train

x_intermediate = np.zeros((8766,17,43,2))
x_intermediate[:,:,:,0] = Tnorm
x_intermediate[:,:,:,1] = Pnorm
x_train_intermediate = x_intermediate[trainInds]
x_test_intermediate = x_intermediate[testInds]

#target
y_train = np.squeeze([F[ii][trainInds[366:]] for ii in range(len(F))])
y_test = np.squeeze([F[ii][testInds[366:]] for ii in range(len(F))])

In [14]:
x = np.zeros((8766-365,365,17,43,2))

for ii in range(len(x_intermediate)-365):
    x[ii] = x_intermediate[ii:ii+365]

In [ ]:
x_train = x[trainInds]
x_test = x[testInds]

In [1]:
type(x)

NameError: name 'x' is not defined

In [6]:
x = np.zeros((8766-365,365,17,43,2))
x = [x_intermediate[ind:ind+365] for ind in range(len(x_intermediate)-365)]

x_train = [x[ind] for ind in trainInds[:-366]]
x_test = [x[ind] for ind in trainInds[:-366]]

In [ ]:
x_train_array = np.asarray(x_train)
x_test_array = np.asarray(x_test)

#x_train = x[trainInds]
#x_test = x[testInds]

In [17]:
type(np.asarray(x_train[:1000]))

numpy.ndarray

In [ ]:
type(np.squeeze(x_test))

In [45]:
np.shape(y_train)

(194, 4748)

In [46]:
import keras
from keras import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense

#CNN model
print('Building model...')
model = Sequential()
model.add(TimeDistributed(
    Conv2D(filters = 8, kernel_size = (3,3), activation='relu',data_format='channels_last'), 
    input_shape=(365,17,43,2)))
model.add(TimeDistributed(MaxPooling2D(pool_size = 2)))
model.add(TimeDistributed(Flatten()))

#LSTM model with time-distributed CNN as input
model.add(LSTM(units = 20, activation='tanh', recurrent_activation='hard_sigmoid', 
               use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
               bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, 
               recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
               kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, implementation=1, 
               return_sequences=False, return_state=False))
model.add(Dense(194, activation = 'sigmoid'))

#compile
print('Compiling model...')
model.compile(loss=keras.losses.MSE,
              optimizer=keras.optimizers.Adam(lr=0.01),
              metrics=['mae'])

Using TensorFlow backend.
/Users/samanderson/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


Building model...
Instructions for updating:
Colocations handled automatically by placer.
Compiling model...


In [47]:
#fit
batch_size = 64
epochs = 3
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 4748 arrays: [array([[[[-4.11131118e-01,  3.09574853e+00],
         [-5.44301395e-01,  3.97064234e+00],
         [-6.69991008e-01,  3.85533140e+00],
         ...,
         [-2.42223720e+00, -4.93264559e-01],
     ...

In [57]:
type(x_train.asarray)

AttributeError: 'list' object has no attribute 'asarray'